#**AI LAW CHAPTER 3. AGENTS**
---

##0.REFERENCE

https://claude.ai/share/c4e700cb-2603-4b72-b3d7-7ef33ec6791f

##1.CONTEXT

**Welcome to Chapter 3: Understanding Multi-Step AI Workflows for Legal Practice**

**What You're About to Experience**

You're about to work with a Google Colab notebook that demonstrates how artificial intelligence can assist with complex legal tasks through coordinated, multi-step workflows. If you've used ChatGPT, Claude, or similar chatbots through a website or mobile app, you already understand the basics: you type a question or request, the AI responds, and you continue the conversation. This notebook takes that familiar experience and transforms it into something fundamentally different, more powerful, and more appropriate for professional legal work.

**Why Casual Chatbot Use Isn't Enough for Lawyers**

When you use a chatbot casually, you might ask it to write an email, explain a concept, or brainstorm ideas. If the response isn't quite right, you refine your prompt and try again. If it makes something up, it's annoying but rarely catastrophic. You're the only person who sees the conversation, there's no record of what happened, and you're not professionally or ethically obligated to verify every claim the AI makes.

Legal practice operates under completely different constraints. When you use AI to help with legal work, you're creating work product that might affect someone's rights, liberty, property, or livelihood. You have ethical duties to your clients including competence, confidentiality, and diligence. You have duties to tribunals including candor and honesty. You operate under rules of professional conduct that can result in discipline, malpractice liability, or sanctions if violated. A hallucinated case citation in a court filing isn't just embarrassing; it can result in sanctions, damage to your reputation, and harm to your client's interests.

This notebook teaches you how to build systems that acknowledge and address these heightened responsibilities. It demonstrates controls, documentation, and governance practices that transform casual AI interaction into professional-grade legal technology.

**What Makes This Chapter Different: Multi-Step Workflows**

In earlier chapters of this course, you learned to use AI for single-task work: summarize this document, draft this email, check this contract for standard provisions. Those are valuable skills, but they represent only a fraction of what modern AI can do. Chapter 3 introduces orchestrated workflows where multiple specialized AI calls work together to accomplish complex objectives that no single prompt could handle.

Think about how a law firm actually works. When a new client matter arrives, you don't assign one lawyer to do everything simultaneously. Instead, different people with different expertise handle different phases. Someone conducts intake and gathers facts. Someone spots legal issues and researches precedents. Someone drafts the primary work product. Someone else reviews it for quality. Perhaps another person performs adversarial review, deliberately trying to find weaknesses. Finally, someone assembles everything into a polished final deliverable. This division of labor produces better results than any single person working alone could achieve.

This notebook implements that same pattern using AI. Instead of asking Claude to "write a legal memo about this situation," which often produces mediocre results, the system breaks the task into discrete steps. An orchestrator function plans the workflow. Specialist functions handle intake, issue spotting, drafting, quality assurance, adversarial review, and final assembly. Each specialist focuses on one thing, does it well, and hands off to the next specialist with notes about what to do next. The result is dramatically higher quality than single-shot generation.

**The Critical Addition: Human Oversight Checkpoints**

The most important feature this chapter introduces isn't the multi-step workflow itself; it's the human approval gates built into the process. After the orchestrator creates a plan, the system pauses for human review and approval before proceeding. After quality assurance identifies potential problems, a human decides whether to continue or stop. These checkpoints ensure that lawyers remain in control rather than allowing AI to operate autonomously.

This matters enormously for professional responsibility. Many bar ethics opinions about AI emphasize that lawyers must supervise AI tools and cannot delegate professional judgment to machines. By building explicit approval points into the workflow, this notebook demonstrates a architecture that keeps humans in the loop at critical decision points. The AI proposes, analyzes, and drafts, but humans approve, direct, and take responsibility.

**Governance and Audit Trails: Proving You Did It Right**

Another fundamental difference between casual chatbot use and professional legal AI is documentation. When you use ChatGPT for personal tasks, the conversation might be saved in your account, but there's no systematic audit trail, no risk logging, no verification tracking, and no archival package you could present to opposing counsel or a disciplinary committee.

This notebook generates comprehensive governance artifacts. Every API call gets logged with cryptographic hashes that prove it happened while protecting confidential content. Every risk the system identifies gets recorded with severity levels and context. Every output includes structured quality checks and explicit statements about what hasn't been verified. At the end, you download a complete audit package with a human-readable guide explaining what everything means.

Why does this matter? Because if you use AI in legal practice, you may eventually need to explain what you did and why. A court might question your methodology. A malpractice insurer might investigate your use of technology. A bar ethics committee might examine whether you properly supervised AI tools. Having a complete, contemporaneous record of what the system did, what risks it flagged, and what verification it recommended gives you defensible documentation of responsible AI use.

**Confidentiality and Privilege: Built-In Protections**

When you use consumer chatbot services, your inputs typically get sent to the company's servers, potentially used for model training, and stored indefinitely. That's acceptable for casual use but completely inappropriate for privileged attorney-client communications or confidential client information.

This notebook implements multiple layers of protection. It includes redaction functions that attempt to remove emails, phone numbers, social security numbers, and addresses before anything goes to the API. It demonstrates minimum-necessary-data principles, sending only information relevant to the task rather than wholesale client files. It logs only cryptographic hashes of prompts rather than actual content. It includes prominent warnings that redaction is imperfect and that you should never paste actual sensitive data into educational tools.

These protections aren't foolproof, and the notebook makes that clear. But they demonstrate the kind of thinking required when building legal AI systems. You must constantly ask: what data am I exposing, where is it going, who can access it, how long is it retained, and what are the privilege implications?

**Why This Chapter Matters for Your Practice**

Chapter 3 represents a inflection point in legal AI capability. Simple single-task AI assistance is already commonplace and relatively low risk. Multi-step agentic workflows are dramatically more powerful but also more complex and potentially risky. They can handle tasks that previously required days of lawyer time. They can coordinate multiple analytical perspectives on the same problem. They can produce work product that, with appropriate human review, rivals what senior associates generate.

But with that power comes responsibility. You need to understand not just how to make these systems work, but how to make them work safely, ethically, and defensibly. You need governance frameworks that scale as your AI usage becomes more sophisticated. You need to develop judgment about when multi-step AI workflows are appropriate and when simpler approaches suffice.

This notebook gives you hands-on experience with production-grade patterns you could actually adapt for practice. It's not a toy demo that skips the hard parts. It includes error handling, retry logic, risk logging, quality checks, and documentation practices you'd need in real use. By working through it, you develop both technical skills and professional judgment about responsible legal AI deployment.

**Your Learning Path Through Ten Sections**

The notebook contains exactly ten sections, each building on the previous ones. You'll start by understanding the concepts, then set up your environment, connect to the AI service, initialize governance systems, learn about confidentiality protections, build the critical JSON communication wrapper, define your agent team and cases, execute complete workflows, practice with your own scenario, and finally package everything for audit and archive.

By the end, you'll have generated multiple complete legal work products through coordinated multi-step workflows, examined the audit trail showing exactly what happened, and downloaded a governance package demonstrating responsible AI use. More importantly, you'll understand the architecture, controls, and thinking required to deploy powerful AI systems in legal practice while maintaining professional and ethical standards.

Welcome to Chapter 3. Let's begin.

##2.LIBRARIES AND ENVIRONMENT

In [9]:
# Cell 2: Installation and Setup

import os
import json
import hashlib
from datetime import datetime
from pathlib import Path
import re

# Create timestamped run directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
RUN_DIR = Path(f"/content/ai_law_ch3_runs/run_{timestamp}")
RUN_DIR.mkdir(parents=True, exist_ok=True)

# Create subdirectories
DELIVERABLES_DIR = RUN_DIR / "deliverables"
DELIVERABLES_DIR.mkdir(exist_ok=True)

print(f"✅ Run directory created: {RUN_DIR}")
print(f"✅ Deliverables directory: {DELIVERABLES_DIR}")

✅ Run directory created: /content/ai_law_ch3_runs/run_20260107_215606
✅ Deliverables directory: /content/ai_law_ch3_runs/run_20260107_215606/deliverables


##3.API KEY AND CLIENT INITIALIZATION

###3.1.OVERVIEW

**Section 3: Connecting to Claude's Brain**

**What This Section Does**

This section establishes the connection between your notebook and Claude, the AI model that will perform the legal analysis work. Think of it as plugging in a power cord before you can use an appliance. Without this connection, nothing else in the notebook can function.

**The API Key: Your Access Credential**

An API key is like a password that proves you have permission to use Claude's services. Anthropic (the company that created Claude) issues these keys to paying customers. When you store your key in Google Colab's secure "Secrets" area, you're putting it in a digital vault that only your notebook can access. This prevents anyone who sees your notebook from stealing your credentials.

**Why We Need a Client Object**

The "client" is a software tool that handles all communication with Claude's servers. When you want Claude to analyze a legal document or generate a memo, the client packages your request properly, sends it over the internet to Anthropic's computers, waits for Claude to process it, and brings the response back to your notebook. Without initializing this client, you'd have no way to talk to Claude.

**The Model Name Matters**

We specify "claude-sonnet-4-5-20250929" as our model because Anthropic offers different versions of Claude, each with different capabilities and costs. Sonnet 4.5 is the most intelligent version available as of this writing, making it ideal for complex legal analysis where accuracy matters most. The numbers at the end indicate the exact release date, which is important for reproducibility. If someone runs your notebook in the future, they'll know exactly which version of Claude you used.

**What Success Looks Like**

When this section runs successfully, you'll see three green checkmarks confirming that your API key loaded, the correct model was selected, and the client initialized properly. If something goes wrong, you'll see a red X with an error message, typically indicating that you forgot to add your API key to Colab Secrets or that you mistyped the secret name.

**Why This Comes Early**

We set up the API connection in Section 3, right after creating folders but before doing any real work, because every subsequent section depends on it. You can't generate legal documents, log API calls, or run workflows without first establishing this fundamental connection. It's the foundation upon which everything else is built.

###3.2.CODE AND IMPLEMENTATION

In [10]:
# Cell 3: API Key and Client Initialization

!pip install anthropic -q

import anthropic
from google.colab import userdata

# Load API key from Colab Secrets
try:
    ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
    os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY

    # Initialize client
    client = anthropic.Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])
    MODEL = "claude-sonnet-4-5-20250929"

    print("✅ API key loaded successfully")
    print(f"✅ Model: {MODEL}")
    print(f"✅ Client initialized")

except Exception as e:
    print(f"❌ Error loading API key: {e}")
    print("Please add ANTHROPIC_API_KEY to Colab Secrets (Settings → Secrets)")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 8.4 MB/s eta 0:00:00
✅ API key loaded successfully
✅ Model: claude-sonnet-4-5-20250929
✅ Client initialized


##4.GOVERNANCE UTILITIES AND INITIAL ARTIFACTS

###4.1.OVERVIEW

**Section 4: Building the Paper Trail**

**What This Section Does**

This section creates the foundational audit files that track every decision and action your AI workflow takes. Think of it as setting up a filing cabinet before you start generating documents. In legal practice, being able to prove what happened, when it happened, and why it happened is absolutely critical. This section ensures you can do exactly that.

**The Run Manifest: Your Case File Cover Sheet**

The run manifest is a single file that captures basic metadata about this particular execution. It records the timestamp, which model you used, what chapter of the course you're working on, and where all the files are stored. If you need to come back six months later and figure out what happened during this run, the manifest is your starting point. It's like the cover page of a case file that tells you what's inside.

**Prompts Log: The Redacted Conversation Record**

The prompts log uses a special format called JSONL, where each line is a separate log entry. Every time your notebook talks to Claude, it writes one line to this file. However, we don't store the actual text of your prompts or Claude's responses, because those might contain confidential client information. Instead, we store cryptographic hashes, which are like digital fingerprints. You can prove a conversation happened and verify its contents haven't been tampered with, but you can't reverse-engineer the original text from the hash. This protects privilege while maintaining accountability.

**Risk Log: Your Early Warning System**

The risk log starts as an empty container ready to collect problems. As your workflow runs, each step evaluates itself for issues like missing facts, potential hallucinations, confidentiality concerns, or unauthorized practice of law. Every risk gets logged here with its severity level, which step detected it, and what the concern was. By the end of your workflow, this file becomes a comprehensive quality control report.

**Pip Freeze: The Reproducibility Safety Net**

This file captures the exact version of every Python library installed in your environment. Software changes constantly, and code that works today might break tomorrow if a library updates. By recording these versions, someone could recreate your exact setup years from now. In legal technology, where you might need to defend your methodology in court or explain it to a bar ethics committee, this level of documentation is invaluable.

**Why We Do This First**

These files must exist before any substantive work begins because they need to capture everything that happens. You can't retroactively create an audit trail. By initializing these governance files in Section 4, we ensure that from the very first API call onward, everything is tracked and documented.

###4.2.CODE AND IMPLEMENTATION

In [11]:
# Cell 4: Governance Utilities and Initial Artifacts

import subprocess

# Initialize governance files

# 1. Run manifest
run_manifest = {
    "run_id": timestamp,
    "run_dir": str(RUN_DIR),
    "created_at": datetime.now().isoformat(),
    "model": MODEL,
    "chapter": "3_agents",
    "author": "Alejandro Reynoso",
    "purpose": "Multi-step legal workflows with human-in-the-loop checkpoints"
}

manifest_path = RUN_DIR / "run_manifest.json"
with open(manifest_path, 'w') as f:
    json.dump(run_manifest, f, indent=2)

# 2. Prompts log (JSONL format)
prompts_log_path = RUN_DIR / "prompts_log.jsonl"
prompts_log_path.touch()

# 3. Risk log
risk_log_path = RUN_DIR / "risk_log.json"
with open(risk_log_path, 'w') as f:
    json.dump({"risks": [], "created_at": datetime.now().isoformat()}, f, indent=2)

# 4. pip freeze for reproducibility
pip_freeze_path = RUN_DIR / "pip_freeze.txt"
try:
    result = subprocess.run(['pip', 'freeze'], capture_output=True, text=True)
    with open(pip_freeze_path, 'w') as f:
        f.write(result.stdout)
except:
    pass

print("✅ Governance artifacts initialized:")
print(f"   📄 {manifest_path}")
print(f"   📄 {prompts_log_path}")
print(f"   📄 {risk_log_path}")
print(f"   📄 {pip_freeze_path}")

✅ Governance artifacts initialized:
   📄 /content/ai_law_ch3_runs/run_20260107_215606/run_manifest.json
   📄 /content/ai_law_ch3_runs/run_20260107_215606/prompts_log.jsonl
   📄 /content/ai_law_ch3_runs/run_20260107_215606/risk_log.json
   📄 /content/ai_law_ch3_runs/run_20260107_215606/pip_freeze.txt


##5.REDACTION AND MINIMUM NECESSARY DATA INTAKE

###5.1.OVERVIEW

**Section 5: Protecting Confidential Information**

**What This Section Does**

This section introduces critical safety tools that attempt to remove sensitive personal information before anything gets sent to Claude or written to your log files. It demonstrates two functions: one that redacts identifiable information, and another that removes unnecessary data fields. Think of this as your first line of defense against accidental confidentiality breaches.

**The Redaction Function: Automated Privacy Protection**

The redaction function scans text looking for patterns that typically indicate sensitive information. It searches for email addresses, phone numbers, social security numbers, and street addresses, then replaces them with placeholder text like "[EMAIL_REDACTED]" or "[PHONE_REDACTED]". This happens automatically before any text goes to Claude's servers or gets written to disk. The function uses pattern matching, which means it looks for structures like three digits, a dash, two digits, a dash, and four digits for social security numbers.

**Why Redaction Is Imperfect**

The notebook displays a very clear warning: redaction is best effort only, not foolproof. Pattern matching catches common formats, but people write information in countless ways. Someone might write "my email is john dot doe at example dot com" instead of using the standard format. The function would miss that. Similarly, sensitive information doesn't always follow predictable patterns. A client's business valuation or proprietary trade secret might look like ordinary text. This is why the warning says never to paste actual sensitive client data into the notebook, even with redaction enabled.

**Minimum Necessary Data: Reducing Exposure**

The second function demonstrates a principle borrowed from healthcare privacy law: only collect and transmit the minimum information necessary to accomplish your task. If you have a dictionary of client information with twenty fields, but only five are relevant to the legal analysis, this function strips out the other fifteen before sending anything to Claude. It returns two things: the cleaned data that will be used, and a list of fields that were removed. This gives you visibility into what's being excluded and confidence that unnecessary exposure is minimized.

**The Demonstration: Learning by Example**

The section includes a live demonstration using fake data. You can see text before redaction (with visible email, phone, and social security number) and after redaction (with placeholders). You also see how the minimum necessary function removes irrelevant fields like shoe size and favorite color while keeping jurisdiction and legal issue information. This hands-on example helps you understand exactly what these functions do before they're used in real workflow steps.

###5.2.CODE AND IMPLEMENTATION

In [12]:
# Cell 5: Redaction and Minimum-Necessary Data Intake

def redact(text):
    """
    Redact potentially sensitive information.
    WARNING: This is best-effort only. Do NOT rely on this for actual sensitive data.
    """
    if not text:
        return text

    redacted = text

    # Email addresses
    redacted = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '[EMAIL_REDACTED]', redacted)

    # Phone numbers (various formats)
    redacted = re.sub(r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b', '[PHONE_REDACTED]', redacted)
    redacted = re.sub(r'\(\d{3}\)\s*\d{3}[-.]?\d{4}', '[PHONE_REDACTED]', redacted)

    # SSN patterns
    redacted = re.sub(r'\b\d{3}-\d{2}-\d{4}\b', '[SSN_REDACTED]', redacted)

    # Street addresses (simple pattern)
    redacted = re.sub(r'\b\d+\s+[A-Z][a-z]+\s+(Street|St|Avenue|Ave|Road|Rd|Boulevard|Blvd|Lane|Ln|Drive|Dr|Court|Ct)\b',
                     '[ADDRESS_REDACTED]', redacted)

    return redacted


def minimum_necessary_intake(facts_dict):
    """
    Helper to remove unnecessary fields before sending to API.
    Returns: (redacted_facts, removed_fields)
    """
    # Define essential fields per domain
    essential_fields = ['jurisdiction', 'legal_issue', 'key_facts', 'desired_outcome', 'timeline']

    removed = []
    cleaned = {}

    for key, value in facts_dict.items():
        if key in essential_fields:
            cleaned[key] = redact(str(value))
        else:
            removed.append(key)

    return cleaned, removed


# Demo with fake data
print("🔒 Redaction Demo\n")

sample_text = """
Client John Doe (john.doe@example.com, 555-123-4567, SSN: 123-45-6789)
lives at 123 Main Street and needs help with a contract dispute.
"""

print("BEFORE REDACTION:")
print(sample_text)
print("\nAFTER REDACTION:")
print(redact(sample_text))

print("\n" + "="*60)
print("\n📋 Minimum-Necessary Intake Demo\n")

sample_facts = {
    "client_name": "John Doe",
    "client_email": "john.doe@example.com",
    "client_phone": "555-123-4567",
    "jurisdiction": "California",
    "legal_issue": "Contract breach",
    "key_facts": ["Contract signed Jan 2024", "Payment due March 2024", "No payment received"],
    "shoe_size": "10.5",
    "favorite_color": "blue"
}

cleaned, removed = minimum_necessary_intake(sample_facts)

print("CLEANED FACTS (sent to API):")
print(json.dumps(cleaned, indent=2))
print(f"\nREMOVED FIELDS: {removed}")
print("\n⚠️  Remember: Redaction is imperfect. Never paste actual sensitive client data.")

🔒 Redaction Demo

BEFORE REDACTION:

Client John Doe (john.doe@example.com, 555-123-4567, SSN: 123-45-6789) 
lives at 123 Main Street and needs help with a contract dispute.


AFTER REDACTION:

Client John Doe ([EMAIL_REDACTED], [PHONE_REDACTED], SSN: [SSN_REDACTED]) 
lives at [ADDRESS_REDACTED] and needs help with a contract dispute.



📋 Minimum-Necessary Intake Demo

CLEANED FACTS (sent to API):
{
  "jurisdiction": "California",
  "legal_issue": "Contract breach",
  "key_facts": "['Contract signed Jan 2024', 'Payment due March 2024', 'No payment received']"
}

REMOVED FIELDS: ['client_name', 'client_email', 'client_phone', 'shoe_size', 'favorite_color']

⚠️  Remember: Redaction is imperfect. Never paste actual sensitive client data.


##6.CLAUDE WRAPPER

###6.1.OVERVIEW

**Section 6: The Critical JSON Wrapper with Prefill Technique**

**What This Section Does**

This is the most technically important section in the entire notebook. It creates the function that actually talks to Claude and forces Claude to return data in a structured, parseable format. Previous versions of this workflow often failed because Claude would wrap its JSON output in conversational text or markdown formatting, breaking the entire system. This section solves that problem using a technique called prefilling.

**The Prefill Technique: Forcing Clean Output**

Here's the key innovation: when you normally ask Claude a question, Claude decides how to format the answer. Sometimes it says "Here's the JSON you requested" and then provides the data, which ruins everything because you can't parse conversational text. The prefill technique works by sending Claude a message that already starts with an opening curly brace. Claude then feels compelled to complete the JSON object rather than add conversational wrapper text. It's like handing someone a sentence that starts "The capital of France is..." and they naturally complete it rather than starting over. This dramatically increases reliability.

**The System Prompt: Setting Strict Rules**

The function includes a system prompt that establishes non-negotiable rules for Claude's behavior. It provides the exact JSON schema that Claude must follow, listing every required field from "task" to "verification_status". It explicitly forbids markdown formatting, conversational text, or anything outside the JSON structure. It also includes critical legal guardrails: never invent citations, always mark verification status as "Not verified", and always include disclaimers. This system prompt acts like jury instructions, telling Claude exactly what's allowed and what's forbidden.

**The Retry Logic: Graceful Failure Handling**

Even with prefill and strict instructions, things can occasionally go wrong. Network issues, API errors, or unexpected responses happen. The function attempts each API call up to three times with progressively stricter temperature settings. Temperature controls randomness in Claude's outputs; lower temperatures produce more deterministic, focused responses. If all three attempts fail, rather than crashing the entire notebook, the function returns a valid JSON object with error information and high-severity risk flags. This ensures your workflow can continue and produce a complete audit trail even when individual steps fail.

**The Smoke Test: Proving It Works**

The section ends with a smoke test that immediately calls the function with simple test data. This proves the prefill technique works before you run the full workflow. You see whether valid JSON comes back on the first attempt and whether the schema validation passes. This immediate feedback gives you confidence that the most critical component of your system is functioning correctly.

###6.2.CODE AND IMPLEMENTATION

In [13]:
# Cell 6: Claude JSON Wrapper with PREFILL TECHNIQUE (CRITICAL)

def call_claude_json_prefill(task_description, facts_list, deliverable_type="memo",
                              word_limit=400, case_id="unknown", step_name="unknown"):
    """
    Call Claude with PREFILL technique to force pure JSON output.

    CRITICAL: Uses assistant message with "{" to force JSON completion.
    """

    # Build minimal system prompt (schema definition)
    system_prompt = """You MUST output ONLY a valid JSON object. Start with { and end with }.

Your output MUST match this EXACT schema:

{
  "task": "string",
  "facts_provided": ["array of strings"],
  "assumptions": ["array of strings"],
  "open_questions": ["array of strings"],
  "work_product": {
    "deliverable_type": "memo|email|outline|checklist|plan",
    "content": "string (must include disclaimer)"
  },
  "quality_checks": {
    "missing_facts_check": ["array"],
    "hallucination_check": ["array"],
    "privilege_confidentiality_check": ["array"],
    "tone_check": ["array"]
  },
  "handoff_notes": {
    "next_agent": "string",
    "what_to_do_next": ["array"],
    "stop_conditions": ["array"]
  },
  "risks": [
    {"type": "confidentiality|privilege|hallucination|missing_facts|unauthorized_practice|overconfidence|prompt_injection|tone|other",
     "severity": "low|medium|high",
     "note": "string"}
  ],
  "verification_status": "Not verified",
  "questions_to_verify": ["array"]
}

RULES:
- verification_status MUST be "Not verified"
- NEVER invent citations, cases, statutes, or authorities
- work_product.content MUST include: "This is a draft only. Not legal advice. Human lawyer review required."
- NO markdown, NO text outside JSON
- NO conversational wrapper
"""

    # Build minimal user prompt (no field-by-field instructions)
    user_prompt = f"""Task: {task_description}

Facts provided:
{json.dumps(facts_list, indent=2)}

Output constraints:
- Deliverable type: {deliverable_type}
- Word limit for work_product.content: {word_limit} words maximum
- Return ONLY JSON matching the schema."""

    max_attempts = 3
    temperatures = [0.1, 0.0, 0.0]

    for attempt in range(max_attempts):
        try:
            temp = temperatures[attempt]

            # PREFILL TECHNIQUE: Add assistant message with "{"
            response = client.messages.create(
                model=MODEL,
                max_tokens=1800,
                temperature=temp,
                system=system_prompt,
                messages=[
                    {"role": "user", "content": user_prompt},
                    {"role": "assistant", "content": "{"} # PREFILL
                ]
            )

            # Extract completion and PREPEND the "{"
            raw_completion = response.content[0].text
            full_json_text = "{" + raw_completion

            # Try to parse
            result = json.loads(full_json_text)

            # Validate required fields
            required_keys = ["task", "facts_provided", "assumptions", "open_questions",
                           "work_product", "quality_checks", "handoff_notes", "risks",
                           "verification_status", "questions_to_verify"]

            missing = [k for k in required_keys if k not in result]
            if missing:
                raise ValueError(f"Missing required keys: {missing}")

            # Auto-add risks if needed
            if not result.get("risks"):
                result["risks"] = []

            # Add metadata
            result["_meta"] = {
                "case_id": case_id,
                "step_name": step_name,
                "attempt": attempt + 1,
                "temperature": temp,
                "timestamp": datetime.now().isoformat()
            }

            # Log to prompts_log.jsonl (redacted)
            log_entry = {
                "timestamp": datetime.now().isoformat(),
                "case_id": case_id,
                "step_name": step_name,
                "prompt_hash": hashlib.sha256(user_prompt.encode()).hexdigest()[:16],
                "response_hash": hashlib.sha256(full_json_text.encode()).hexdigest()[:16],
                "attempt": attempt + 1,
                "success": True
            }
            with open(prompts_log_path, 'a') as f:
                f.write(json.dumps(log_entry) + '\n')

            # Log risks
            if result.get("risks"):
                with open(risk_log_path, 'r') as f:
                    risk_data = json.load(f)
                for risk in result["risks"]:
                    risk_data["risks"].append({
                        **risk,
                        "case_id": case_id,
                        "step_name": step_name,
                        "timestamp": datetime.now().isoformat()
                    })
                with open(risk_log_path, 'w') as f:
                    json.dump(risk_data, f, indent=2)

            return result

        except json.JSONDecodeError as e:
            if attempt == max_attempts - 1:
                # Final attempt failed - return valid error JSON
                error_result = {
                    "task": task_description,
                    "facts_provided": facts_list,
                    "assumptions": ["JSON parsing failed after 3 attempts"],
                    "open_questions": ["Unable to complete task due to parsing error"],
                    "work_product": {
                        "deliverable_type": deliverable_type,
                        "content": f"ERROR: Could not generate valid JSON output. This is a draft only. Not legal advice. Human lawyer review required. Error: {str(e)[:100]}"
                    },
                    "quality_checks": {
                        "missing_facts_check": ["N/A - error state"],
                        "hallucination_check": ["N/A - error state"],
                        "privilege_confidentiality_check": ["N/A - error state"],
                        "tone_check": ["N/A - error state"]
                    },
                    "handoff_notes": {
                        "next_agent": "Human lawyer",
                        "what_to_do_next": ["Review error", "Retry manually"],
                        "stop_conditions": ["System error"]
                    },
                    "risks": [
                        {
                            "type": "other",
                            "severity": "high",
                            "note": f"JSON_PARSE_ERROR: {str(e)[:100]}"
                        }
                    ],
                    "verification_status": "Not verified",
                    "questions_to_verify": ["All output invalid due to error"],
                    "_meta": {
                        "case_id": case_id,
                        "step_name": step_name,
                        "attempt": attempt + 1,
                        "error": True
                    }
                }
                return error_result
        except Exception as e:
            if attempt == max_attempts - 1:
                error_result = {
                    "task": task_description,
                    "facts_provided": facts_list,
                    "assumptions": ["API call failed"],
                    "open_questions": ["Unable to complete task due to API error"],
                    "work_product": {
                        "deliverable_type": deliverable_type,
                        "content": f"ERROR: API call failed. This is a draft only. Not legal advice. Human lawyer review required. Error: {str(e)[:100]}"
                    },
                    "quality_checks": {
                        "missing_facts_check": ["N/A - error state"],
                        "hallucination_check": ["N/A - error state"],
                        "privilege_confidentiality_check": ["N/A - error state"],
                        "tone_check": ["N/A - error state"]
                    },
                    "handoff_notes": {
                        "next_agent": "Human lawyer",
                        "what_to_do_next": ["Review error", "Check API status"],
                        "stop_conditions": ["API error"]
                    },
                    "risks": [
                        {
                            "type": "other",
                            "severity": "high",
                            "note": f"API_ERROR: {str(e)[:100]}"
                        }
                    ],
                    "verification_status": "Not verified",
                    "questions_to_verify": ["All output invalid due to error"],
                    "_meta": {
                        "case_id": case_id,
                        "step_name": step_name,
                        "attempt": attempt + 1,
                        "error": True
                    }
                }
                return error_result

# SMOKE TEST
print("🧪 Running prefill smoke test...\n")

test_result = call_claude_json_prefill(
    task_description="Test: Generate a simple legal memo outline",
    facts_list=["Test fact 1: Client needs contract review", "Test fact 2: California law applies"],
    deliverable_type="outline",
    word_limit=100,
    case_id="smoke_test",
    step_name="validation"
)

if test_result and "task" in test_result and "_meta" in test_result:
    if test_result["_meta"].get("error"):
        print("❌ SMOKE TEST FAILED - Error in response")
        print(f"Error: {test_result['risks'][0]['note']}")
    else:
        print("✅ SMOKE TEST PASSED")
        print(f"   - Valid JSON returned on attempt {test_result['_meta']['attempt']}")
        print(f"   - Schema validation: PASS")
        print(f"   - Prefill wrapper: READY")
else:
    print("❌ SMOKE TEST FAILED - Invalid structure")

print("\n" + "="*60)
print("✅ call_claude_json_prefill() ready for use")

🧪 Running prefill smoke test...

✅ SMOKE TEST PASSED
   - Valid JSON returned on attempt 1
   - Schema validation: PASS
   - Prefill wrapper: READY

✅ call_claude_json_prefill() ready for use


##7.ORCHESTRATOR AND AGENT DEFINITIONS

###7.1.OVERVIEW

**Section 7: Building the Agent Team and Defining Cases**

**What This Section Does**

This section creates the specialized agent functions that will work together to handle complex legal tasks, and it defines four realistic mini-cases that will test the system. Think of this as assembling your legal team where each member has a specific expertise, then briefing them on the cases they'll handle. Nothing executes yet; this section is pure setup and planning.

**Understanding Agent Functions: Specialized Roles**

Each agent function represents a distinct phase of legal work. The orchestrator plans the overall workflow strategy. The intake agent identifies missing information and formulates follow-up questions. The issue spotter analyzes legal problems and potential claims. The drafting agent creates the primary work product. The quality assurance agent reviews for completeness and accuracy. The red team agent deliberately looks for weaknesses and counterarguments. Finally, the final assembly agent integrates everything into a polished deliverable. Each function calls the Claude wrapper from Section 6 but with different task descriptions and expectations.

**Why Specialization Matters**

Breaking complex legal work into specialized steps serves multiple purposes. First, it prevents cognitive overload. Asking Claude to simultaneously draft, quality check, and red team in one step produces inferior results compared to doing each separately. Second, it creates natural checkpoints where human lawyers can review and approve before proceeding. Third, it generates a detailed audit trail showing exactly which step produced which output. Fourth, if one step fails, you know precisely where the problem occurred rather than having to debug a monolithic process.

**The Four Mini-Cases: Testing Across Domains**

The section defines four cases spanning different legal domains to prove the workflow handles diverse situations. The criminal case involves bail and early defense strategy. The regulatory case deals with federal agency rulemaking and comment procedures. The international case addresses cross-border commercial disputes with choice of law complications. The teaching case tackles academic policy development. Each case includes concrete facts, specified deliverable types, and a complete workflow plan listing all seven steps that will execute.

**Simplified Prompts: Learning from Past Failures**

Notice what's not in this section: lengthy instructions about JSON field requirements. Earlier versions failed because they included detailed schema instructions in every prompt, which confused Claude and triggered conversational explanation mode. Now the system prompt in Section 6 handles schema enforcement, while these case definitions stay minimal. Each case just provides facts, specifies a word limit, and states the desired deliverable type. This simplicity is deliberate and critical to reliability.

###7.2.CODE AND IMPLEMENTATION

In [14]:
# Cell 7: Orchestrator and Agent Definitions + Mini-Cases (CRITICAL)

# --- AGENT STEP FUNCTIONS ---

def orchestrator_plan(case_facts, case_id):
    """Generate a workflow plan for the case."""
    return call_claude_json_prefill(
        task_description="Orchestrator: Create a step-by-step workflow plan for this legal matter",
        facts_list=case_facts,
        deliverable_type="plan",
        word_limit=300,
        case_id=case_id,
        step_name="orchestrator_plan"
    )

def intake_agent(case_facts, case_id):
    """Initial intake and information gathering."""
    return call_claude_json_prefill(
        task_description="Intake Agent: Identify missing information and formulate questions for client",
        facts_list=case_facts,
        deliverable_type="checklist",
        word_limit=250,
        case_id=case_id,
        step_name="intake"
    )

def issue_spotter(case_facts, case_id):
    """Identify legal issues and potential claims/defenses."""
    return call_claude_json_prefill(
        task_description="Issue Spotter: Identify all legal issues, potential claims, and defenses",
        facts_list=case_facts,
        deliverable_type="memo",
        word_limit=350,
        case_id=case_id,
        step_name="issue_spotter"
    )

def drafting_agent(case_facts, deliverable_type, case_id):
    """Draft primary work product."""
    return call_claude_json_prefill(
        task_description=f"Drafting Agent: Create a {deliverable_type} based on the provided facts",
        facts_list=case_facts,
        deliverable_type=deliverable_type,
        word_limit=400,
        case_id=case_id,
        step_name="drafting"
    )

def qa_agent(case_facts, draft_content, case_id):
    """Quality assurance review."""
    qa_facts = case_facts + [f"Draft to review: {draft_content[:200]}..."]
    return call_claude_json_prefill(
        task_description="QA Agent: Review draft for completeness, accuracy, and quality issues",
        facts_list=qa_facts,
        deliverable_type="checklist",
        word_limit=300,
        case_id=case_id,
        step_name="qa"
    )

def redteam_agent(case_facts, draft_content, case_id):
    """Red team adversarial review."""
    redteam_facts = case_facts + [f"Draft to challenge: {draft_content[:200]}..."]
    return call_claude_json_prefill(
        task_description="Red Team Agent: Identify weaknesses, gaps, and potential counterarguments",
        facts_list=redteam_facts,
        deliverable_type="memo",
        word_limit=300,
        case_id=case_id,
        step_name="redteam"
    )

def final_assembly(case_facts, all_outputs, case_id):
    """Assemble final deliverable incorporating all feedback."""
    assembly_facts = case_facts + [
        f"Outputs to integrate: {len(all_outputs)} prior steps completed"
    ]
    return call_claude_json_prefill(
        task_description="Final Assembly: Create polished final deliverable incorporating all prior work",
        facts_list=assembly_facts,
        deliverable_type="memo",
        word_limit=450,
        case_id=case_id,
        step_name="final_assembly"
    )


# --- MINI-CASE DEFINITIONS ---

MINI_CASES = [
    {
        "case_id": "criminal_001",
        "domain": "Criminal Defense",
        "facts": [
            "Client arrested on state drug possession charges (California)",
            "First offense, no prior criminal record",
            "Arrested during traffic stop without warrant",
            "Client has stable employment and family ties in community",
            "Bail hearing scheduled in 48 hours"
        ],
        "deliverable_type": "memo",
        "workflow_steps": ["orchestrator_plan", "intake", "issue_spotter", "drafting", "qa", "redteam", "final_assembly"]
    },
    {
        "case_id": "regulatory_001",
        "domain": "Regulatory/Administrative",
        "facts": [
            "Federal agency issued NPRM affecting client's financial services business",
            "Comment period ends in 30 days",
            "Proposed rule changes capital requirements and reporting obligations",
            "Client operates in multiple states",
            "Potential compliance costs estimated at $2M annually"
        ],
        "deliverable_type": "memo",
        "workflow_steps": ["orchestrator_plan", "intake", "issue_spotter", "drafting", "qa", "redteam", "final_assembly"]
    },
    {
        "case_id": "international_001",
        "domain": "International Commercial",
        "facts": [
            "Contract dispute between US company (Delaware) and UK supplier",
            "Contract silent on choice of law and forum",
            "Breach relates to delivery delays and quality issues",
            "Contract value: $5M over 3 years",
            "Both parties interested in preserving business relationship if possible"
        ],
        "deliverable_type": "memo",
        "workflow_steps": ["orchestrator_plan", "intake", "issue_spotter", "drafting", "qa", "redteam", "final_assembly"]
    },
    {
        "case_id": "teaching_001",
        "domain": "Teaching/Academic",
        "facts": [
            "Law school considering AI policy for student submissions",
            "Concerns about plagiarism, academic integrity, and skill development",
            "Need to balance innovation with educational objectives",
            "ABA accreditation requirements must be met",
            "Policy should cover exams, papers, and legal writing assignments"
        ],
        "deliverable_type": "outline",
        "workflow_steps": ["orchestrator_plan", "intake", "issue_spotter", "drafting", "qa", "redteam", "final_assembly"]
    }
]

print("✅ Agent functions defined:")
print("   - orchestrator_plan()")
print("   - intake_agent()")
print("   - issue_spotter()")
print("   - drafting_agent()")
print("   - qa_agent()")
print("   - redteam_agent()")
print("   - final_assembly()")

print(f"\n✅ {len(MINI_CASES)} mini-cases loaded:")
for case in MINI_CASES:
    print(f"   - {case['case_id']}: {case['domain']} ({len(case['workflow_steps'])} steps)")

✅ Agent functions defined:
   - orchestrator_plan()
   - intake_agent()
   - issue_spotter()
   - drafting_agent()
   - qa_agent()
   - redteam_agent()
   - final_assembly()

✅ 4 mini-cases loaded:
   - criminal_001: Criminal Defense (7 steps)
   - regulatory_001: Regulatory/Administrative (7 steps)
   - international_001: International Commercial (7 steps)
   - teaching_001: Teaching/Academic (7 steps)


##8.EXECUTION

###8.1.OVERVIEW

**Section 8: Executing the Complete Workflow**

**What This Section Does**

This is where everything comes together and actually runs. Section 8 takes the four mini-cases defined earlier and executes the complete seven-step workflow for each one, tracking progress, handling errors, logging risks, saving outputs, and generating a comprehensive summary. This is the engine room of the entire notebook where theory becomes practice and you see the multi-step agent system in action.

**Progressive Execution with Visual Feedback**

The section processes cases sequentially, showing you exactly where it is at every moment. You see "Case 1 of 4" then within that case "Step 2 of 7" so you always know progress. Each step prints a status indicator: a green checkmark for success or a red X for failure with a brief error message. This real-time feedback is crucial because the full execution might take several minutes, and you need to know the system is working rather than frozen.

**Error Handling: Continuing Despite Problems**

The section wraps every case and every step in error protection. If the issue spotter fails for the criminal case, the workflow notes the failure but continues to the drafting step. This resilience ensures you get as much output as possible rather than one early failure killing the entire run. Each error gets logged with context about which case and step failed. By the end, you have a complete picture of what succeeded, what failed, and where problems occurred.

**Human Approval Gates: Simulated Checkpoints**

Real multi-step workflows need human oversight at critical transition points. After the orchestrator creates a plan, a lawyer should review and approve it before work begins. After quality assurance identifies issues, someone should decide whether to proceed or stop. This section simulates those approval gates using boolean variables set to true. In production use, these would be actual prompts asking the lawyer to review and approve. The simulation demonstrates where these checkpoints belong and why they matter for responsible AI use.

**Comprehensive Output Generation**

For each step in each case, the section saves two files: a complete JSON file with all structured data including quality checks and risk flags, and a plain text file containing just the work product content. After all steps complete, it creates consolidated bundle files that integrate everything. This gives you both granular per-step outputs for detailed review and integrated final deliverables for practical use.

**The Statistics Dashboard: Quantifying Performance**

As execution proceeds, the section tracks cases attempted versus successful, steps attempted versus successful, total API calls made, and total risks logged. At the end, it displays a formatted table showing each case with a status icon, steps completed, and highest risk severity. This dashboard gives you immediate insight into system performance and helps identify patterns like whether certain types of cases or steps fail more frequently.

###8.2.CODE AND IMPLEMENTATION

In [15]:
# Cell 8: Execute Multi-Step Workflows for All Cases

import time

# Statistics
stats = {
    "cases_attempted": 0,
    "cases_success": 0,
    "cases_failed": 0,
    "steps_attempted": 0,
    "steps_success": 0,
    "steps_failed": 0,
    "api_calls": 0,
    "total_risks_logged": 0
}

case_results = []

print("🚀 Starting multi-step workflow execution for all cases\n")
print("="*70)

for case_idx, case in enumerate(MINI_CASES, 1):
    case_id = case["case_id"]
    domain = case["domain"]
    facts = case["facts"]
    deliverable_type = case["deliverable_type"]
    steps = case["workflow_steps"]

    print(f"\n[Case {case_idx}/{len(MINI_CASES)}] {case_id} - {domain}")
    print("-" * 70)

    stats["cases_attempted"] += 1

    # Create case deliverables directory
    case_dir = DELIVERABLES_DIR / case_id
    case_dir.mkdir(exist_ok=True)

    case_status = "success"
    steps_completed = 0
    all_outputs = {}
    highest_risk_severity = "low"

    try:
        # Save case facts
        with open(case_dir / "case_facts.json", 'w') as f:
            json.dump({"case_id": case_id, "domain": domain, "facts": facts}, f, indent=2)

        # Execute workflow steps
        for step_idx, step_name in enumerate(steps, 1):
            print(f"  [Step {step_idx}/{len(steps)}] {step_name}...", end=" ")
            stats["steps_attempted"] += 1
            stats["api_calls"] += 1

            try:
                # Execute appropriate agent function
                if step_name == "orchestrator_plan":
                    result = orchestrator_plan(facts, case_id)
                elif step_name == "intake":
                    result = intake_agent(facts, case_id)
                elif step_name == "issue_spotter":
                    result = issue_spotter(facts, case_id)
                elif step_name == "drafting":
                    result = drafting_agent(facts, deliverable_type, case_id)
                elif step_name == "qa":
                    draft_content = all_outputs.get("drafting", {}).get("work_product", {}).get("content", "No draft available")
                    result = qa_agent(facts, draft_content, case_id)
                elif step_name == "redteam":
                    draft_content = all_outputs.get("drafting", {}).get("work_product", {}).get("content", "No draft available")
                    result = redteam_agent(facts, draft_content, case_id)
                elif step_name == "final_assembly":
                    result = final_assembly(facts, all_outputs, case_id)
                else:
                    result = None

                if result:
                    all_outputs[step_name] = result
                    steps_completed += 1
                    stats["steps_success"] += 1

                    # Save step output
                    with open(case_dir / f"{step_name}_output.json", 'w') as f:
                        json.dump(result, f, indent=2)

                    # Save work product as text
                    if "work_product" in result and "content" in result["work_product"]:
                        with open(case_dir / f"{step_name}_draft.txt", 'w') as f:
                            f.write(result["work_product"]["content"])

                    # Track highest risk severity
                    for risk in result.get("risks", []):
                        stats["total_risks_logged"] += 1
                        if risk["severity"] == "high":
                            highest_risk_severity = "high"
                        elif risk["severity"] == "medium" and highest_risk_severity != "high":
                            highest_risk_severity = "medium"

                    print("✅")

                    # Simulate human approval gates
                    if step_name == "orchestrator_plan":
                        APPROVE_PLAN = True  # Simulated approval
                        if not APPROVE_PLAN:
                            print("    ⛔ Plan approval gate: REJECTED")
                            case_status = "failed"
                            break

                    if step_name == "qa":
                        APPROVE_CONTINUE = True  # Simulated approval
                        if not APPROVE_CONTINUE:
                            print("    ⛔ QA approval gate: REJECTED")
                            case_status = "failed"
                            break

                else:
                    print("❌ (no result)")
                    stats["steps_failed"] += 1

            except Exception as e:
                print(f"❌ Error: {str(e)[:50]}")
                stats["steps_failed"] += 1
                # Continue to next step

        # Create final bundle
        if all_outputs:
            final_bundle = {
                "case_id": case_id,
                "domain": domain,
                "facts": facts,
                "steps_completed": steps_completed,
                "total_steps": len(steps),
                "outputs": all_outputs,
                "highest_risk_severity": highest_risk_severity,
                "timestamp": datetime.now().isoformat()
            }

            with open(case_dir / "final_bundle.json", 'w') as f:
                json.dump(final_bundle, f, indent=2)

            # Create consolidated text output
            with open(case_dir / "final_bundle.txt", 'w') as f:
                f.write(f"CASE: {case_id} - {domain}\n")
                f.write("="*70 + "\n\n")
                f.write("FACTS:\n")
                for fact in facts:
                    f.write(f"  - {fact}\n")
                f.write("\n" + "="*70 + "\n\n")

                for step_name, output in all_outputs.items():
                    f.write(f"STEP: {step_name.upper()}\n")
                    f.write("-"*70 + "\n")
                    if "work_product" in output and "content" in output["work_product"]:
                        f.write(output["work_product"]["content"])
                    f.write("\n\n")

            stats["cases_success"] += 1
        else:
            case_status = "failed"
            stats["cases_failed"] += 1

        case_results.append({
            "case_id": case_id,
            "domain": domain,
            "status": case_status,
            "steps_completed": steps_completed,
            "total_steps": len(steps),
            "highest_risk": highest_risk_severity
        })

    except Exception as e:
        print(f"  ❌ Case-level error: {str(e)[:100]}")
        stats["cases_failed"] += 1
        case_results.append({
            "case_id": case_id,
            "domain": domain,
            "status": "failed",
            "steps_completed": steps_completed,
            "total_steps": len(steps),
            "highest_risk": "high"
        })

# Print summary
print("\n" + "="*70)
print("📊 EXECUTION SUMMARY")
print("="*70)

print(f"\nCases: {stats['cases_success']}/{stats['cases_attempted']} successful")
print(f"Steps: {stats['steps_success']}/{stats['steps_attempted']} successful")
print(f"API Calls: {stats['api_calls']}")
print(f"Total Risks Logged: {stats['total_risks_logged']}")

print("\n" + "-"*70)
print(f"{'Case ID':<18} {'Domain':<25} {'Status':<8} {'Steps':<12} {'Risk':<8}")
print("-"*70)

for result in case_results:
    status_icon = "✅" if result["status"] == "success" else "❌"
    steps_text = f"{result['steps_completed']}/{result['total_steps']}"
    print(f"{result['case_id']:<18} {result['domain']:<25} {status_icon:<8} {steps_text:<12} {result['highest_risk']:<8}")

print("\n✅ All deliverables saved to:", DELIVERABLES_DIR)

🚀 Starting multi-step workflow execution for all cases


[Case 1/4] criminal_001 - Criminal Defense
----------------------------------------------------------------------
  [Step 1/7] orchestrator_plan... ✅
  [Step 2/7] intake... ✅
  [Step 3/7] issue_spotter... ✅
  [Step 4/7] drafting... ✅
  [Step 5/7] qa... ✅
  [Step 6/7] redteam... ✅
  [Step 7/7] final_assembly... ✅

[Case 2/4] regulatory_001 - Regulatory/Administrative
----------------------------------------------------------------------
  [Step 1/7] orchestrator_plan... ✅
  [Step 2/7] intake... ✅
  [Step 3/7] issue_spotter... ✅
  [Step 4/7] drafting... ✅
  [Step 5/7] qa... ✅
  [Step 6/7] redteam... ✅
  [Step 7/7] final_assembly... ✅

[Case 3/4] international_001 - International Commercial
----------------------------------------------------------------------
  [Step 1/7] orchestrator_plan... ✅
  [Step 2/7] intake... ✅
  [Step 3/7] issue_spotter... ✅
  [Step 4/7] drafting... ✅
  [Step 5/7] qa... ✅
  [Step 6/7] redteam... ✅
  [Step 

##9.OWN EXAMPLES

###9.1.OVERVIEW

**Section 9: Your Turn to Practice**

**What This Section Does**

This section shifts from demonstration to practice by letting you input your own hypothetical legal scenario and run it through the same multi-step workflow. It's designed as a safe learning exercise where you can experiment with the system using made-up facts, see how it responds to different types of cases, and understand the workflow from a user perspective rather than just watching preset examples.

**Interactive Input with Safety Reminders**

The section begins with prominent warnings reminding you not to paste actual sensitive client data. It then prompts you for three pieces of information: a brief description of the legal situation, the relevant jurisdiction, and key facts separated by semicolons. In the demonstration version, these use preset values you can modify, but in live use you would type directly into input boxes. This interactivity makes the learning experience personal and relevant to your practice areas.

**Automatic Redaction Before Processing**

Before your input goes anywhere near Claude or gets written to files, it passes through the redaction function from Section 5. You see a summary showing which fields were removed by the minimum necessary filter and how many fact items will actually be sent to the API. This demonstrates the privacy protection workflow in action with your own data, reinforcing the principle that you should minimize exposure even with hypothetical scenarios.

**Workflow Type Selection: Tailoring the Approach**

Different legal domains require different analytical approaches. The section offers five workflow options: criminal defense, regulatory administrative, international commercial, teaching academic, and a custom employment category. Your selection determines which specialist agents emphasize which aspects of analysis. A criminal workflow focuses heavily on procedural rights and immediate client protection, while a regulatory workflow emphasizes compliance timeline and comment strategy. This demonstrates how the same underlying agent architecture adapts to domain-specific needs.

**Shortened Four-Step Workflow: Faster Learning**

Rather than running all seven steps from the demonstration cases, your exercise uses a condensed four-step version: orchestrator planning, issue spotting, drafting, and final assembly. This provides faster feedback while still demonstrating the core multi-step coordination pattern. You still see the orchestrator create a plan, the system pause for simulated approval, specialists do their work, and the final assembly integrate everything. The shorter sequence makes experimentation more practical during a learning session.

**Deliverables in Your Own Folder**

All outputs from your exercise save to a separate "user_case" folder, keeping your work distinct from the demonstration cases. You get the same quality of output: JSON files with complete structured data and text files with readable work products. This lets you compare your results against the preset examples and see how the system handles your specific fact patterns and jurisdictional context.

###9.2.CODE AND IMPLEMENTATION

In [16]:
# Cell 9: User Exercise - Custom Case Workflow

print("👤 USER EXERCISE: Run Your Own Case Through the Workflow")
print("="*70)
print("\n⚠️  IMPORTANT: Do NOT paste actual sensitive client data.")
print("Use hypothetical facts for learning purposes only.\n")

# Simulate user input (in actual use, would use input())
# For demo purposes, we'll use preset values that can be modified

print("Please provide information about your hypothetical case:\n")

# In live use, uncomment these lines:
# user_scenario = input("Describe the legal situation (1-2 sentences): ")
# user_jurisdiction = input("Jurisdiction (e.g., California, New York): ")
# user_facts_input = input("Key facts (separate with semicolons): ")

# Demo values (modify these for testing):
user_scenario = "Client needs help with employment discrimination claim"
user_jurisdiction = "New York"
user_facts_input = "Client terminated after pregnancy announcement; worked for company 3 years; no prior disciplinary issues; company has 75 employees; termination occurred 2 weeks after announcement"

print(f"\nScenario: {user_scenario}")
print(f"Jurisdiction: {user_jurisdiction}")
print(f"Facts: {user_facts_input}")

# Workflow type selection
print("\nSelect workflow type:")
print("1. Criminal Defense")
print("2. Regulatory/Administrative")
print("3. International Commercial")
print("4. Teaching/Academic")
print("5. Employment (custom)")

# For demo: workflow_type = input("Enter number (1-5): ")
workflow_type = "5"  # Employment custom

# Process input
user_facts_raw = {
    "scenario": user_scenario,
    "jurisdiction": user_jurisdiction,
    "raw_facts": user_facts_input
}

# Redact
redacted_facts, removed_fields = minimum_necessary_intake(user_facts_raw)
user_facts_list = [
    f"Jurisdiction: {user_jurisdiction}",
    f"Situation: {user_scenario}"
] + [f.strip() for f in user_facts_input.split(";") if f.strip()]

print("\n" + "="*70)
print("🔒 REDACTION SUMMARY")
print("="*70)
print(f"Fields removed: {removed_fields if removed_fields else 'None'}")
print(f"Facts to be sent: {len(user_facts_list)} items")

# Create user case directory
user_case_dir = DELIVERABLES_DIR / "user_case"
user_case_dir.mkdir(exist_ok=True)

print("\n" + "="*70)
print("🚀 EXECUTING WORKFLOW (Shortened 4-step version)")
print("="*70)

# Shortened workflow for user exercise
user_workflow = ["orchestrator_plan", "issue_spotter", "drafting", "final_assembly"]

user_outputs = {}
user_case_id = "user_exercise"

for step_idx, step_name in enumerate(user_workflow, 1):
    print(f"\n[Step {step_idx}/{len(user_workflow)}] {step_name}...", end=" ")

    try:
        if step_name == "orchestrator_plan":
            result = orchestrator_plan(user_facts_list, user_case_id)
        elif step_name == "issue_spotter":
            result = issue_spotter(user_facts_list, user_case_id)
        elif step_name == "drafting":
            result = drafting_agent(user_facts_list, "memo", user_case_id)
        elif step_name == "final_assembly":
            result = final_assembly(user_facts_list, user_outputs, user_case_id)
        else:
            result = None

        if result:
            user_outputs[step_name] = result

            # Save
            with open(user_case_dir / f"{step_name}_output.json", 'w') as f:
                json.dump(result, f, indent=2)

            if "work_product" in result and "content" in result["work_product"]:
                with open(user_case_dir / f"{step_name}_draft.txt", 'w') as f:
                    f.write(result["work_product"]["content"])

            print("✅")

            # Approval gate simulation
            if step_name == "orchestrator_plan":
                APPROVE_USER_PLAN = True
                if not APPROVE_USER_PLAN:
                    print("    ⛔ Plan rejected. Workflow stopped.")
                    break

    except Exception as e:
        print(f"❌ Error: {str(e)[:50]}")

# Save user bundle
if user_outputs:
    user_bundle = {
        "case_id": user_case_id,
        "scenario": user_scenario,
        "jurisdiction": user_jurisdiction,
        "facts": user_facts_list,
        "outputs": user_outputs,
        "timestamp": datetime.now().isoformat()
    }

    with open(user_case_dir / "user_bundle.json", 'w') as f:
        json.dump(user_bundle, f, indent=2)

    print("\n" + "="*70)
    print("✅ USER EXERCISE COMPLETE")
    print("="*70)
    print(f"Deliverables saved to: {user_case_dir}")
    print(f"Total outputs: {len(user_outputs)}")
else:
    print("\n❌ User exercise failed to produce outputs")

👤 USER EXERCISE: Run Your Own Case Through the Workflow

⚠️  IMPORTANT: Do NOT paste actual sensitive client data.
Use hypothetical facts for learning purposes only.

Please provide information about your hypothetical case:


Scenario: Client needs help with employment discrimination claim
Jurisdiction: New York
Facts: Client terminated after pregnancy announcement; worked for company 3 years; no prior disciplinary issues; company has 75 employees; termination occurred 2 weeks after announcement

Select workflow type:
1. Criminal Defense
2. Regulatory/Administrative
3. International Commercial
4. Teaching/Academic
5. Employment (custom)

🔒 REDACTION SUMMARY
Fields removed: ['scenario', 'raw_facts']
Facts to be sent: 7 items

🚀 EXECUTING WORKFLOW (Shortened 4-step version)

[Step 1/4] orchestrator_plan... ✅

[Step 2/4] issue_spotter... ✅

[Step 3/4] drafting... 

KeyboardInterrupt: 

##10.ARTIFACTS

###10.1.OVERVIEW

**Section 10: Packaging Everything for Audit and Archive**

**What This Section Does**

This final section creates a comprehensive audit package containing every file generated during the workflow execution, writes a detailed human-readable guide explaining what's in the package, compresses everything into a downloadable zip file, and provides a final checklist confirming all governance artifacts exist. This is where raw execution results transform into a professional, archivable audit trail suitable for bar ethics review or firm quality control processes.

**The Audit README: Your Guide for Future Review**

The section generates a multi-page text document that explains what every file in the package means and how to interpret it. This readme assumes the reader might be reviewing the package months or years later, possibly someone who wasn't involved in the original execution. It explains the purpose of the prompts log, why only hashes are stored instead of full text, what the risk severity levels mean, and how to navigate the deliverables folder structure. It also includes all the critical disclaimers about draft status, verification requirements, and lawyer responsibility.

**The Review Methodology: Step-by-Step Instructions**

The readme doesn't just describe files; it prescribes a specific review sequence. Start with the manifest to understand run parameters. Then examine the risk log to identify high-severity concerns. Next review the actual work products in the deliverables folder. Check the prompts log to verify proper logging occurred. Finally, work through a risk mitigation checklist that includes verifying citations, checking privilege considerations, and confirming human review happened. This structured approach ensures consistent, thorough audits regardless of who performs them.

**File Inventory: Complete Transparency**

Before creating the zip file, the section lists every single file in the package with its size in kilobytes. This inventory serves two purposes. First, it gives you immediate visibility into what's being archived. Second, it creates a permanent record of package contents that could prove completeness if challenged later. You can see at a glance that you have outputs for all four cases, that the risk log exists, and that governance files are present.

**The ZIP Bundle: Portable Archive**

The section compresses the entire run directory into a single zip file with a timestamped filename. This makes the package easy to download from Google Colab to your local machine, attach to emails, store in document management systems, or archive for long-term retention. The compression also makes file transfer faster and storage more efficient. The section reports the final zip file size so you know whether it's small enough to email or needs alternative transfer methods.

**Final Checklist: Verification Before Completion**

The very last output is a checklist with green checkmarks or red X marks showing whether each required governance artifact exists. This final verification step catches any problems before you rely on the package. If the risk log is missing or the deliverables folder is empty, you know immediately rather than discovering it later when you need to reference the audit trail.

###10.2.CODE AND IMPLEMENTATION

In [17]:
# Cell 10: Create Audit Package and Download Bundle

import shutil

print("📦 Creating audit package and download bundle")
print("="*70)

# Create AUDIT_README.txt
audit_readme_path = RUN_DIR / "AUDIT_README.txt"

with open(audit_readme_path, 'w') as f:
    f.write("""
================================================================================
AI FOR LAWYERS - CHAPTER 3 (LEVEL 3: AGENTS)
AUDIT PACKAGE README
================================================================================

This package contains a complete audit trail for a multi-step agentic workflow
execution using Claude (Anthropic) for legal task automation.

PACKAGE CONTENTS:
================================================================================

1. run_manifest.json
   - Run metadata (timestamp, model, chapter, author)
   - Directory structure information

2. prompts_log.jsonl
   - Redacted log of all API calls (JSONL format, one entry per line)
   - Each entry contains: timestamp, case_id, step_name, prompt_hash,
     response_hash, attempt number, success status
   - Full prompts/responses NOT stored (only hashes for verification)

3. risk_log.json
   - Comprehensive list of ALL risks flagged during execution
   - Each risk includes: type, severity, note, case_id, step_name, timestamp
   - Risk types: confidentiality, privilege, hallucination, missing_facts,
     unauthorized_practice, overconfidence, prompt_injection, tone, other

4. deliverables/ folder
   - One subdirectory per case (e.g., criminal_001/, regulatory_001/)
   - Each case directory contains:
     * case_facts.json - Original facts provided
     * <step_name>_output.json - Full JSON output per workflow step
     * <step_name>_draft.txt - Human-readable work product per step
     * final_bundle.json - Consolidated outputs from all steps
     * final_bundle.txt - Consolidated text document

5. pip_freeze.txt
   - Python package versions for reproducibility

6. AUDIT_README.txt (this file)
   - Human-readable guide to package contents

================================================================================
IMPORTANT DISCLAIMERS
================================================================================

⚠️  ALL OUTPUTS ARE DRAFTS ONLY
- Every work product requires lawyer review
- No attorney-client relationship created
- Not legal advice to you or anyone
- Verification status: "Not verified" on all outputs

⚠️  CONFIDENTIALITY & PRIVILEGE
- Redaction applied (imperfect - best effort only)
- Do not rely on redaction for sensitive data
- Consider privilege implications before using AI

⚠️  MODEL LIMITATIONS
- May hallucinate facts, citations, or authorities
- All legal references marked "Not verified"
- Human lawyer MUST verify all substantive content

⚠️  WORKFLOW RESPONSIBILITY
- "Agent" = workflow pattern, NOT autonomous AI
- Lawyer remains responsible for all decisions
- Human approval gates simulated in demo

================================================================================
HOW TO REVIEW THIS PACKAGE
================================================================================

1. START WITH: run_manifest.json
   - Understand run parameters and timestamp

2. REVIEW RISKS: risk_log.json
   - Check for high-severity risks
   - Understand risk distribution across cases/steps

3. EXAMINE OUTPUTS: deliverables/<case_id>/
   - Review final_bundle.txt for each case
   - Check per-step outputs for quality and consistency
   - Verify all disclaimers present

4. AUDIT TRAIL: prompts_log.jsonl
   - Verify all API calls logged
   - Check success rates and retry patterns
   - Confirm redaction applied (only hashes stored)

5. RISK MITIGATION CHECKLIST:
   [ ] All high-severity risks addressed
   [ ] Legal citations independently verified
   [ ] Confidential information properly handled
   [ ] Privilege considerations documented
   [ ] Human lawyer review completed
   [ ] Disclaimers present in all outputs
   [ ] Jurisdiction-specific rules checked

================================================================================
TECHNICAL DETAILS
================================================================================

Platform: Google Colab
SDK: anthropic (Python)
Model: claude-sonnet-4-5-20250929
Chapter: 3 (Level 3: Agents)
Author: Alejandro Reynoso, Chief Scientist DEFI CAPITAL RESEARCH
        External Lecturer, Judge Business School Cambridge

Workflow Pattern: Orchestrator + Specialist Agents
- Orchestrator: Plans workflow steps
- Specialists: Intake, Issue Spotter, Drafting, QA, Red Team, Final Assembly
- Human Gates: Approval checkpoints between major phases

JSON Schema: Strict enforcement via prefill technique
- All outputs match predefined schema
- verification_status: "Not verified" (mandatory)
- Quality checks included in every output
- Handoff notes for multi-step coordination

================================================================================
CONTACT & FEEDBACK
================================================================================

For questions about this educational tool:
- Review course materials for Chapter 3
- Consult with legal tech specialists at your firm
- Consider ethics opinions in your jurisdiction

This is educational software for demonstration purposes.
Production use requires additional controls, testing, and legal review.

================================================================================
END OF AUDIT README
================================================================================
""")

print(f"✅ Created: {audit_readme_path}")

# List all files in package
print("\n📋 PACKAGE CONTENTS:")
print("-" * 70)

all_files = list(RUN_DIR.rglob("*"))
for fpath in sorted(all_files):
    if fpath.is_file():
        rel_path = fpath.relative_to(RUN_DIR)
        size_kb = fpath.stat().st_size / 1024
        print(f"  {rel_path} ({size_kb:.1f} KB)")

print(f"\nTotal files: {len([f for f in all_files if f.is_file()])}")

# Create ZIP bundle
zip_path = Path(f"/content/ai_law_ch3_bundle_{timestamp}.zip")
print(f"\n📦 Creating ZIP bundle: {zip_path.name}")

shutil.make_archive(
    str(zip_path.with_suffix('')),
    'zip',
    RUN_DIR
)

zip_size_mb = zip_path.stat().st_size / (1024 * 1024)

print(f"✅ ZIP bundle created: {zip_path}")
print(f"   Size: {zip_size_mb:.2f} MB")

# Final checklist
print("\n" + "="*70)
print("✅ FINAL PACKAGE CHECKLIST")
print("="*70)

checklist = [
    ("run_manifest.json", (RUN_DIR / "run_manifest.json").exists()),
    ("prompts_log.jsonl", (RUN_DIR / "prompts_log.jsonl").exists()),
    ("risk_log.json", (RUN_DIR / "risk_log.json").exists()),
    ("AUDIT_README.txt", (RUN_DIR / "AUDIT_README.txt").exists()),
    ("pip_freeze.txt", (RUN_DIR / "pip_freeze.txt").exists()),
    ("deliverables/ folder", (RUN_DIR / "deliverables").exists()),
    ("ZIP bundle", zip_path.exists())
]

for item, exists in checklist:
    status = "✅" if exists else "❌"
    print(f"{status} {item}")

print("\n" + "="*70)
print("🎉 CHAPTER 3 WORKFLOW COMPLETE")
print("="*70)
print(f"\n📂 Run directory: {RUN_DIR}")
print(f"📦 Download bundle: {zip_path}")
print("\n⚠️  Remember: All outputs require lawyer review. Not legal advice.")

📦 Creating audit package and download bundle
✅ Created: /content/ai_law_ch3_runs/run_20260107_215606/AUDIT_README.txt

📋 PACKAGE CONTENTS:
----------------------------------------------------------------------
  AUDIT_README.txt (5.2 KB)
  deliverables/criminal_001/case_facts.json (0.3 KB)
  deliverables/criminal_001/drafting_draft.txt (0.2 KB)
  deliverables/criminal_001/drafting_output.json (1.6 KB)
  deliverables/criminal_001/final_assembly_draft.txt (2.5 KB)
  deliverables/criminal_001/final_assembly_output.json (7.6 KB)
  deliverables/criminal_001/final_bundle.json (44.3 KB)
  deliverables/criminal_001/final_bundle.txt (11.2 KB)
  deliverables/criminal_001/intake_draft.txt (1.3 KB)
  deliverables/criminal_001/intake_output.json (6.5 KB)
  deliverables/criminal_001/issue_spotter_draft.txt (1.8 KB)
  deliverables/criminal_001/issue_spotter_output.json (7.0 KB)
  deliverables/criminal_001/orchestrator_plan_draft.txt (1.5 KB)
  deliverables/criminal_001/orchestrator_plan_output.json (

##11.CONCLUSIONS

**Conclusion: Understanding the Complete Pipeline**

**The Journey You Just Completed**

You've worked through a sophisticated ten-section pipeline that transforms raw AI capability into a defensible, auditable, professional-grade legal workflow system. Each section played a specific role in building toward the final goal: demonstrating that multi-step AI workflows can assist with complex legal tasks while maintaining the controls, documentation, and human oversight that professional responsibility requires. Let's trace the complete path from beginning to end to solidify your understanding of how all the pieces fit together.

**Foundation: Concepts and Infrastructure**

The pipeline began with Section 1 establishing the conceptual framework. You learned what multi-step agentic workflows mean, why they represent a capability leap beyond simple chatbot interactions, and what new risks they introduce. This wasn't just theoretical background; it was essential context for understanding every decision made in subsequent sections. You learned that "agent" means a workflow pattern, not autonomous AI, and that lawyers remain responsible despite the system's sophistication.

Section 2 created the physical infrastructure by establishing a timestamped run directory and subdirectories for deliverables. This might seem mundane, but it's foundational. Without organized file storage established upfront, later sections would have nowhere to save outputs. The timestamp ensures every run gets its own isolated space, preventing different executions from overwriting each other's results.

Section 3 connected you to Claude by loading your API key securely and initializing the client object. This is the moment the notebook gained the ability to actually communicate with AI. Without this connection, everything else would be theoretical. The section verified the connection worked and confirmed which model version you're using, ensuring reproducibility.

**Governance Framework: Building Accountability**

Section 4 initialized the governance artifacts that make this system auditable rather than just functional. It created the run manifest documenting metadata, the prompts log ready to record every API interaction, the risk log prepared to capture every concern flagged during execution, and the pip freeze file enabling reproducibility. These files transform informal AI experimentation into documented professional work with accountability built in from the start.

Section 5 introduced confidentiality protections through redaction and minimum-necessary-data functions. This section acknowledged a hard truth: AI systems require data to function, but legal professionals handle sensitive information that shouldn't be freely transmitted. The redaction demonstration showed both the capability and limitations of automated privacy protection, emphasizing that technology assists but cannot replace human judgment about what information to share.

**The Critical Technical Core**

Section 6 implemented the most technically sophisticated component: the JSON wrapper function using the prefill technique. This section solved the reliability problem that plagued earlier systems where Claude would wrap JSON in conversational text, breaking the parser. By forcing Claude to complete a JSON object already started with an opening brace, the system achieved dramatically higher reliability. The section also implemented retry logic, error handling, automatic risk flagging, and comprehensive logging. This wrapper is the beating heart of the entire system; every substantive AI interaction flows through it.

Section 7 defined the cast of characters and the cases they'll handle. It created seven specialist agent functions, each focused on one phase of legal work: orchestration, intake, issue spotting, drafting, quality assurance, adversarial review, and final assembly. It also defined four mini-cases spanning criminal, regulatory, international, and academic domains. This section was pure setup, but essential setup. It established the division of labor that makes multi-step workflows effective and provided concrete test scenarios representing real practice areas.

**Execution: Theory Becomes Practice**

Section 8 brought everything together by executing the complete seven-step workflow for all four cases. This is where you saw the system actually work. The orchestrator planned each case's approach. Specialists performed their designated tasks in sequence. Human approval gates simulated checkpoints where lawyers would review and authorize proceeding. Quality assurance and red team agents provided complementary perspectives on draft work products. Final assembly integrated everything into polished deliverables.

The section also demonstrated professional-grade error handling. When individual steps failed, the system logged the failure and continued rather than crashing. Statistics tracking showed success rates across cases and steps. The final summary table provided immediate visibility into what worked and what didn't. Every output saved to appropriately named files in organized folders, and every risk got logged with context about which case and step identified it.

**Personalization and Packaging**

Section 9 shifted from demonstration to practice by letting you input your own hypothetical scenario. This section proved the system handles novel situations, not just preset examples. It applied the same confidentiality protections, ran a condensed workflow, and generated deliverables in a separate user folder. This hands-on experience transformed you from observer to participant, cementing understanding through practice.

Section 10 completed the pipeline by packaging everything for long-term use. It generated a comprehensive audit readme explaining what every file means and how to review the package. It inventoried all files with sizes. It compressed everything into a downloadable zip bundle. It provided a final checklist verifying all governance artifacts exist. This section ensured that the work product you generated during execution becomes a portable, archivable, auditable package suitable for professional quality control processes.

**The Complete Picture**

The pipeline flows logically from concepts to infrastructure to connections to governance to protection to core functionality to agent definitions to execution to practice to packaging. Each section depends on previous sections and enables subsequent ones. Nothing is extraneous; every component serves the goal of demonstrating that powerful AI workflows can coexist with professional responsibility when designed thoughtfully.

You now understand not just how to run the notebook, but why each section exists, what it contributes, and how it fits into the larger architecture. This understanding enables you to adapt these patterns to your own practice, evaluate commercial legal AI products more critically, and discuss AI governance with colleagues and ethics committees from a position of informed expertise.

The pipeline you've traced represents the future of legal practice: powerful AI assistance operating within robust professional controls.